In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn.neighbors
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn import svm

In [2]:
cd C:\Users\Alon\Desktop\Alon\School\4th year\SemesterB\Data science\ex4\

C:\Users\Alon\Desktop\Alon\School\4th year\SemesterB\Data science\ex4


In [3]:
df = pd.read_csv('data/train.csv')

In [4]:
df['Gender'] = df['Gender'].fillna( df['Gender'].dropna().mode().values[0] )
df['Married'] = df['Married'].fillna( df['Married'].dropna().mode().values[0] )
df['Dependents'] = df['Dependents'].fillna( df['Dependents'].dropna().mode().values[0] )
df['Self_Employed'] = df['Self_Employed'].fillna( df['Self_Employed'].dropna().mode().values[0] )
df['LoanAmount'] = df['LoanAmount'].fillna( df['LoanAmount'].dropna().mean() )
df['Loan_Amount_Term'] = df['Loan_Amount_Term'].fillna( df['Loan_Amount_Term'].dropna().mode().values[0] )
df['Credit_History'] = df['Credit_History'].fillna( df['Credit_History'].dropna().mode().values[0] )
df['Dependents'] = df['Dependents'].str.rstrip('+')
df['Gender'] = df['Gender'].map({'Female':0,'Male':1}).astype(np.int)
df['Married'] = df['Married'].map({'No':0, 'Yes':1}).astype(np.int)
df['Education'] = df['Education'].map({'Not Graduate':0, 'Graduate':1}).astype(np.int)
df['Self_Employed'] = df['Self_Employed'].map({'No':0, 'Yes':1}).astype(np.int)
df['Loan_Status'] = df['Loan_Status'].map({'N':0, 'Y':1}).astype(np.int)
df['Dependents'] = df['Dependents'].astype(np.int)

In [5]:
from sklearn.preprocessing import LabelEncoder
var_mod = list(df.columns.values)
le = LabelEncoder()
for i in var_mod:
    df[i] = le.fit_transform(df[i].astype(str))

In [6]:
X,y  = df.iloc[:, 1:-1], df.iloc[:, -1]

In [7]:
X= pd.get_dummies(X)


In [8]:
from sklearn.preprocessing import StandardScaler
slc= StandardScaler()
X_train_std = slc.fit_transform(X)

In [9]:
#neigh = KNeighborsClassifier(n_neighbors=4)
from sklearn import svm
clf = svm.SVC(kernel='linear',C=1.5)
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=4)

In [10]:
neigh.fit(X_train_std, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=4, p=2,
           weights='uniform')

In [11]:
clf.fit(X_train_std, y)

SVC(C=1.5, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [12]:
dtest = pd.read_csv('data/test.csv')
from sklearn.preprocessing import LabelEncoder
#var_mod = list(dtest.columns.values)
#le = LabelEncoder()
#for i in var_mod:
 #  dtest[i] = le.fit_transform(dtest[i].astype(str))

In [13]:
dtest = pd.read_csv('data/test.csv')
dtest['Dependents'] = dtest['Dependents'].str.rstrip('+')
dtest['Gender'] = dtest['Gender'].fillna( dtest['Gender'].dropna().mode().values[0])
dtest['Dependents'] = dtest['Dependents'].fillna( dtest['Dependents'].dropna().mode().values[0]).astype(np.int)
dtest['Self_Employed'] = dtest['Self_Employed'].fillna( dtest['Self_Employed'].dropna().mode().values[0])
dtest['LoanAmount'] = dtest['LoanAmount'].fillna( dtest['LoanAmount'].dropna().mode().values[0])
dtest['Loan_Amount_Term'] = dtest['Loan_Amount_Term'].fillna( dtest['Loan_Amount_Term'].dropna().mode().values[0])
dtest['Credit_History'] = dtest['Credit_History'].fillna( dtest['Credit_History'].dropna().mode().values[0] )
dtest['Gender'] = dtest['Gender'].map({'Female':0,'Male':1})
dtest['Married'] = dtest['Married'].map({'No':0, 'Yes':1}).astype(np.int)
dtest['Education'] = dtest['Education'].map({'Not Graduate':0, 'Graduate':1}).astype(np.int)
dtest['Self_Employed'] = dtest['Self_Employed'].map({'No':0, 'Yes':1})

In [14]:
X_test = dtest.iloc[:,1:]

In [15]:
X_test= pd.get_dummies(X_test)
X_test['Property_Area']=X_test['Property_Area_Rural']+X_test['Property_Area_Semiurban']+X_test['Property_Area_Urban']
X_test.drop('Property_Area_Rural', axis=1, inplace=True)
X_test.drop('Property_Area_Semiurban', axis=1, inplace=True)
X_test.drop('Property_Area_Urban', axis=1, inplace=True)

In [16]:
X_test_std = slc.transform(X_test)
y_test_pred = clf.predict(X_test_std)
y_test_pred2 = neigh.predict(X_test_std)
dtest['Loan_Status'] = y_test_pred

In [17]:
df_final = dtest.drop(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'Property_Area'], axis=1)
df_final['Loan_Status'] = df_final['Loan_Status'].map({0:'N', 1:'Y'})
df_final.to_csv('my_submission1.csv', index=False)

In [18]:
dtest['Loan_Status'] = y_test_pred2
df_final = dtest.drop(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'Property_Area'], axis=1)
df_final['Loan_Status'] = df_final['Loan_Status'].map({0:'N', 1:'Y'})
df_final.to_csv('my_submission2.csv', index=False)